# Use API as an agent

In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv
from langchain.agents import initialize_agent,AgentType
from langchain_tavily import TavilySearch
load_dotenv()

True

In [2]:
model= ChatGroq(
model="llama-3.3-70b-versatile",
temperature=0.5
)

search_tool=TavilySearch(search_depth="basic",)


In [4]:
tools=[search_tool]

agent=initialize_agent(tools,model,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True,return_intermediate_steps=True)

agent.invoke("What is the weather in Barisal Division")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search",
  "action_input": {
    "query": "Barisal Division weather",
    "include_domains": [],
    "exclude_domains": [],
    "search_depth": "basic",
    "include_images": false,
    "time_range": null,
    "topic": "general",
    "include_favicon": false,
    "start_date": null,
    "end_date": null
  }
}
```

Observation: {'query': 'Barisal Division weather', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Barisal Division, Bangladesh', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Barisal', 'region': '', 'country': 'Bangladesh', 'lat': 22.7, 'lon': 90.3667, 'tz_id': 'Asia/Dhaka', 'localtime_epoch': 1756887789, 'localtime': '2025-09-03 14:23'}, 'current': {'last_updated_epoch': 1756887300, 'last_updated': '2025-09-03 14:15', 'temp_c': 31.7, 'temp_f': 89.0, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/

{'input': 'What is the weather in Barisal Division',
 'output': 'The current weather in Barisal Division is partly cloudy with a temperature of 31.7°C and a feels-like temperature of 37.7°C. The humidity is 65% and the wind is blowing at 29.5 km/h from the SSE direction.',
 'intermediate_steps': [(AgentAction(tool='tavily_search', tool_input={'query': 'Barisal Division weather', 'include_domains': [], 'exclude_domains': [], 'search_depth': 'basic', 'include_images': False, 'time_range': None, 'topic': 'general', 'include_favicon': False, 'start_date': None, 'end_date': None}, log='Action:\n```\n{\n  "action": "tavily_search",\n  "action_input": {\n    "query": "Barisal Division weather",\n    "include_domains": [],\n    "exclude_domains": [],\n    "search_depth": "basic",\n    "include_images": false,\n    "time_range": null,\n    "topic": "general",\n    "include_favicon": false,\n    "start_date": null,\n    "end_date": null\n  }\n}\n```\n'),
   {'query': 'Barisal Division weather',


# Make your own Agent

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/react")  # this is the ReAct agent prompt template

In [10]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [11]:
from langchain.agents import create_react_agent,AgentExecutor
from langchain.tools import Tool
from langsmith import Client, wrappers
from dotenv import load_dotenv
load_dotenv()


def add_exclamation(query: str) -> str:
    return query + "!!!"

exclaim_tool = Tool(
    name="Exclaim",
    description="Make any phrase exciting by appending exclamation points. Input is a plain string.",
    func=add_exclamation,
)


from langchain_ollama import ChatOllama

llm = ChatOllama(model="gemma3:4b",temperature=.4)



In [12]:
tools=[exclaim_tool]

prompt = hub.pull("hwchase17/react")  # this is the ReAct agent prompt template

# 4. Create the ReAct agent
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# 5. Wrap in an executor to run it
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=3,)

result = agent_executor.invoke({"input": "Take the phrase 'hello NOOBIESS' and make it exciting"})
print(result["output"])




> Entering new AgentExecutor chain...
Thought: The question asks me to make the phrase 'hello NOOBIESS' exciting by adding exclamation points.
Action: Exclaim
Action Input: hello NOOBIESShello NOOBIESS!!!Thought: I have successfully added exclamation points to the phrase.
Final Answer: hello NOOBIESS!!!

> Finished chain.
hello NOOBIESS!!!


In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
print("Tracing:", os.getenv("LANGSMITH_TRACING"))
print("Project:", os.getenv("LANGSMITH_PROJECT"))
print("API Key starts with:", os.getenv("LANGSMITH_API_KEY")[:4])


Tracing: true
Project: test
API Key starts with: lsv2


In [7]:
model=ChatOllama(model="gemma3:4b",temperature=.4)
model.invoke("Hello, how are you?")

AIMessage(content='I’m doing well, thank you for asking! As a large language model, I don’t really *feel* in the way humans do, but my systems are running smoothly and I’m ready to chat. 😊 \n\nHow are *you* doing today? Is there anything you’d like to talk about or anything I can help you with?', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-09-01T13:51:51.0665652Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11411776900, 'load_duration': 4006855500, 'prompt_eval_count': 15, 'prompt_eval_duration': 870248700, 'eval_count': 75, 'eval_duration': 6533037300, 'model_name': 'gemma3:4b'}, id='run--a13f848e-2cfe-4736-addd-0fccff7735e7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 75, 'total_tokens': 90})